## Question 3
What movies are the easiest to make? How many crew members does it require, and how much will it cost? Are the easiest movies to make the most successful?

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stat
import plotly.express as px
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
from sklearn import preprocessing

In [2]:
gross_df = pd.read_csv('./Data/grossdf.csv')
staffing_df = pd.read_csv('./Data/staffingdf.csv')
full_ratings_df = pd.read_csv('./Data/ratingsdf.csv')

In [3]:
display(gross_df.head())
display(staffing_df.head())
full_ratings_df.head()

,Unnamed: 0,title,production_budget_in_mil,dom_gross_mil
0,0,Toy Story 3,200.0,415.0
1,1,Alice in Wonderland (2010),NaN,334.2
2,2,Harry Potter and the Deathly Hallows Part 1,NaN,296.0
3,3,Inception,160.0,292.6
4,4,Shrek Forever After,165.0,238.7


,Unnamed: 0,index,sum_of_nconst
0,0,tt2447934,10
1,1,tt1984208,10
2,2,tt10097958,10
3,3,tt7091418,10
4,4,tt5722530,10


,Unnamed: 0,popularity,title,genres,avg_rating,total_votes
0,0,33.533,Harry Potter and the Deathly Hallows: Part 1,"Adventure,Fantasy,Mystery",7.70,436318.0
1,1,28.734,How to Train Your Dragon,"Action,Adventure,Animation",7.90,618909.0
2,2,28.515,Iron Man 2,"Action,Adventure,Sci-Fi",6.90,670058.0
3,3,28.005,Toy Story,NaN,7.90,10174.0
4,5,27.920,Inception,"Action,Adventure,Sci-Fi",8.55,1863252.0


I've realized here that I cleaned my data in a manner that won't help me answer this question, as my df that has the data for how many crew members each movie had doesn't have a corresponding value for the name of the movie, so I have to do some more cleaning for this question.

In [4]:
imdb_name_df = pd.read_csv('./zippedData/unzippedData/imdb.name.basics.csv')
imdb_title_principals_df = pd.read_csv('./zippedData/unzippedData/imdb.title.principals.csv')
imdb_title_rating_df = pd.read_csv('./zippedData/unzippedData/imdb.title.ratings.csv')
imdb_title_basic_df = pd.read_csv('./zippedData/unzippedData/imdb.title.basics.csv')
tn_movie_budgets_df = pd.read_csv('./zippedData/unzippedData/tn.movie_budgets.csv')

In [5]:
list_of_dfs = [gross_df, staffing_df, full_ratings_df,
              imdb_name_df, imdb_title_principals_df,
              imdb_title_rating_df, tn_movie_budgets_df,
              imdb_title_basic_df]

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

for n, df in enumerate(list_of_dfs):
    print(get_df_name(df))
    print(df.shape)
    display(df.head())

gross_df
(7930, 4)


,Unnamed: 0,title,production_budget_in_mil,dom_gross_mil
0,0,Toy Story 3,200.0,415.0
1,1,Alice in Wonderland (2010),NaN,334.2
2,2,Harry Potter and the Deathly Hallows Part 1,NaN,296.0
3,3,Inception,160.0,292.6
4,4,Shrek Forever After,165.0,238.7


staffing_df
(143454, 3)


,Unnamed: 0,index,sum_of_nconst
0,0,tt2447934,10
1,1,tt1984208,10
2,2,tt10097958,10
3,3,tt7091418,10
4,4,tt5722530,10


full_ratings_df
(87008, 6)


...

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


tn_movie_budgets_df
(5782, 6)


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


imdb_title_basic_df
(146144, 6)


,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [6]:
staffing_df['tconst'] = staffing_df['index']
staffing_df.head()

,Unnamed: 0,index,sum_of_nconst,tconst
0,0,tt2447934,10,tt2447934
1,1,tt1984208,10,tt1984208
2,2,tt10097958,10,tt10097958
3,3,tt7091418,10,tt7091418
4,4,tt5722530,10,tt5722530


In [7]:
staff_titles_df = pd.merge(staffing_df, imdb_title_basic_df, how = 'left', on = 'tconst')
staff_titles_df.head()

,Unnamed: 0,index,sum_of_nconst,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,0,tt2447934,10,tt2447934,"White as Milk, Red as Blood","Bianca come il latte, rossa come il sangue",2013,102.0,"Comedy,Drama"
1,1,tt1984208,10,tt1984208,Moby Dick,Mo-bi-dik,2011,112.0,Action
2,2,tt10097958,10,tt10097958,Gripped: Climbing Killer Pillar,Gripped: Climbing Killer Pillar,2019,NaN,"Action,Adventure"
3,3,tt7091418,10,tt7091418,Crossroad,Crossroad,2017,168.0,Drama
4,4,tt5722530,10,tt5722530,Maligno,Maligno,2016,87.0,"Horror,Mystery"


In [8]:
display(staff_titles_df.shape)
staffing_df.shape

(143454, 9)

(143454, 4)

In [9]:
staff_titles_df['title'] = staff_titles_df['primary_title']
staff_titles_df.drop(['Unnamed: 0', 'index', 'start_year', 'runtime_minutes', 'genres', 'primary_title', 'original_title'],
                    axis = 1, inplace = True)
staff_titles_df.head()

,sum_of_nconst,tconst,title
0,10,tt2447934,"White as Milk, Red as Blood"
1,10,tt1984208,Moby Dick
2,10,tt10097958,Gripped: Climbing Killer Pillar
3,10,tt7091418,Crossroad
4,10,tt5722530,Maligno


In [10]:
df_adjust = [staff_titles_df, full_ratings_df, gross_df]

for df in df_adjust:
    df['title'] = df['title'].apply(lambda x: x.replace(":", ""))
    df['title'] = df['title'].apply(lambda x: x.replace(",", ""))
    df['title'] = df['title'].apply(lambda x: x.replace(".", ""))
    df['title'] = df['title'].apply(lambda x: x.replace("'", ""))
    df['title'] = df['title'].apply(lambda x: x.replace("!", ""))
    df['title'] = df['title'].apply(lambda x: x.replace("-", ""))
    df['title'] = df['title'].apply(lambda x: x.replace(";", ""))
    df['title'] = df['title'].apply(lambda x: x.title())
    df['title'] = df['title'].apply(lambda x: x.strip())

In [11]:
staff_ratings_df = pd.merge(full_ratings_df, staff_titles_df,
                           how = "inner", on = 'title')
display(staff_ratings_df.head())
staff_ratings_df.shape

,Unnamed: 0,popularity,title,genres,avg_rating,total_votes,sum_of_nconst,tconst
0,0,33.533,Harry Potter And The Deathly Hallows Part 1,"Adventure,Fantasy,Mystery",7.70,436318.0,10,tt0926084
1,1,28.734,How To Train Your Dragon,"Action,Adventure,Animation",7.90,618909.0,10,tt0892769
2,2,28.515,Iron Man 2,"Action,Adventure,Sci-Fi",6.90,670058.0,10,tt1228705
3,5,27.920,Inception,"Action,Adventure,Sci-Fi",8.55,1863252.0,10,tt1375666
4,6,26.691,Percy Jackson & The Olympians The Lightning Thief,"Adventure,Family,Fantasy",6.00,169337.0,10,tt0814255


(104342, 8)

In [12]:
final_df = pd.merge(gross_df, staff_ratings_df, how = 'left', on = 'title')
display(final_df.head())
final_df.shape

,Unnamed: 0_x,title,production_budget_in_mil,dom_gross_mil,Unnamed: 0_y,popularity,genres,avg_rating,total_votes,sum_of_nconst,tconst
0,0,Toy Story 3,200.0,415.0,8.0,24.445,"Adventure,Animation,Comedy",8.00,690558.0,10.0,tt0435761
1,1,Alice In Wonderland (2010),NaN,334.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Harry Potter And The Deathly Hallows Part 1,NaN,296.0,0.0,33.533,"Adventure,Fantasy,Mystery",7.70,436318.0,10.0,tt0926084
3,3,Inception,160.0,292.6,5.0,27.920,"Action,Adventure,Sci-Fi",8.55,1863252.0,10.0,tt1375666
4,4,Shrek Forever After,165.0,238.7,51.0,15.041,"Adventure,Animation,Comedy",6.20,171375.0,10.0,tt0892791


(20848, 11)

In [13]:
final_df.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis = 1, inplace = True)
final_df.head()

,title,production_budget_in_mil,dom_gross_mil,popularity,genres,avg_rating,total_votes,sum_of_nconst,tconst
0,Toy Story 3,200.0,415.0,24.445,"Adventure,Animation,Comedy",8.00,690558.0,10.0,tt0435761
1,Alice In Wonderland (2010),NaN,334.2,NaN,NaN,NaN,NaN,NaN,NaN
2,Harry Potter And The Deathly Hallows Part 1,NaN,296.0,33.533,"Adventure,Fantasy,Mystery",7.70,436318.0,10.0,tt0926084
3,Inception,160.0,292.6,27.920,"Action,Adventure,Sci-Fi",8.55,1863252.0,10.0,tt1375666
4,Shrek Forever After,165.0,238.7,15.041,"Adventure,Animation,Comedy",6.20,171375.0,10.0,tt0892791


In [14]:
for column in final_df.columns:
    print(f'{column.title()} has: {final_df[column].isna().sum()} NaN values out of {final_df.shape[0]}, for a fraction of {round(100*(final_df[column].isna().sum()/final_df.shape[0]), 3)}%.')

Title has: 0 NaN values out of 20848, for a fraction of 0.0%.
Production_Budget_In_Mil has: 3748 NaN values out of 20848, for a fraction of 17.978%.
Dom_Gross_Mil has: 76 NaN values out of 20848, for a fraction of 0.365%.
Popularity has: 6541 NaN values out of 20848, for a fraction of 31.375%.
Genres has: 4317 NaN values out of 20848, for a fraction of 20.707%.
Avg_Rating has: 4095 NaN values out of 20848, for a fraction of 19.642%.
Total_Votes has: 4095 NaN values out of 20848, for a fraction of 19.642%.
Sum_Of_Nconst has: 4095 NaN values out of 20848, for a fraction of 19.642%.
Tconst has: 4095 NaN values out of 20848, for a fraction of 19.642%.


In [15]:
final_df['title'].duplicated().value_counts()

True     13061
False     7787
Name: title, dtype: int64

In [16]:
final_df['bool'] = final_df['title'].duplicated()
final_df.head()

,title,production_budget_in_mil,dom_gross_mil,popularity,genres,avg_rating,total_votes,sum_of_nconst,tconst,bool
0,Toy Story 3,200.0,415.0,24.445,"Adventure,Animation,Comedy",8.00,690558.0,10.0,tt0435761,False
1,Alice In Wonderland (2010),NaN,334.2,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Harry Potter And The Deathly Hallows Part 1,NaN,296.0,33.533,"Adventure,Fantasy,Mystery",7.70,436318.0,10.0,tt0926084,False
3,Inception,160.0,292.6,27.920,"Action,Adventure,Sci-Fi",8.55,1863252.0,10.0,tt1375666,False
4,Shrek Forever After,165.0,238.7,15.041,"Adventure,Animation,Comedy",6.20,171375.0,10.0,tt0892791,False


In [17]:
q = """SELECT title, production_budget_in_mil AS prod_budget_mil,
       dom_gross_mil, popularity, genres, avg_rating, sum_of_nconst AS staff
       FROM final_df
       WHERE bool == False"""

true_final_df = pysqldf(q)
display(true_final_df.head())
true_final_df.shape

,title,prod_budget_mil,dom_gross_mil,popularity,genres,avg_rating,staff
0,Toy Story 3,200.0,415.0,24.445,"Adventure,Animation,Comedy",8.00,10.0
1,Alice In Wonderland (2010),NaN,334.2,NaN,None,NaN,NaN
2,Harry Potter And The Deathly Hallows Part 1,NaN,296.0,33.533,"Adventure,Fantasy,Mystery",7.70,10.0
3,Inception,160.0,292.6,27.920,"Action,Adventure,Sci-Fi",8.55,10.0
4,Shrek Forever After,165.0,238.7,15.041,"Adventure,Animation,Comedy",6.20,10.0


(7787, 7)

In [18]:
for column in true_final_df.columns:
    print(f'{column.title()} has: {true_final_df[column].isna().sum()} NaN values out of {true_final_df.shape[0]}, for a fraction of {round(100*(true_final_df[column].isna().sum()/true_final_df.shape[0]), 3)}%.')

Title has: 0 NaN values out of 7787, for a fraction of 0.0%.
Prod_Budget_Mil has: 2148 NaN values out of 7787, for a fraction of 27.584%.
Dom_Gross_Mil has: 26 NaN values out of 7787, for a fraction of 0.334%.
Popularity has: 5177 NaN values out of 7787, for a fraction of 66.483%.
Genres has: 4124 NaN values out of 7787, for a fraction of 52.96%.
Avg_Rating has: 4067 NaN values out of 7787, for a fraction of 52.228%.
Staff has: 4067 NaN values out of 7787, for a fraction of 52.228%.


Here we can see that Popularity, Genre, Avg_Rating, and Staff have more than half their values representing as NaNs. Since the values I need for this question are the production budget and the staff, I'll need to condense the values yet again.

In [19]:
true_final_df.drop('popularity', axis = 1, inplace = True)
true_final_df.head()

,title,prod_budget_mil,dom_gross_mil,genres,avg_rating,staff
0,Toy Story 3,200.0,415.0,"Adventure,Animation,Comedy",8.00,10.0
1,Alice In Wonderland (2010),NaN,334.2,None,NaN,NaN
2,Harry Potter And The Deathly Hallows Part 1,NaN,296.0,"Adventure,Fantasy,Mystery",7.70,10.0
3,Inception,160.0,292.6,"Action,Adventure,Sci-Fi",8.55,10.0
4,Shrek Forever After,165.0,238.7,"Adventure,Animation,Comedy",6.20,10.0


In [20]:
true_final_df['rating_bool'] = true_final_df['avg_rating'].duplicated()
true_final_df['staff_bool'] = true_final_df['staff'].duplicated()
true_final_df.head()

,title,prod_budget_mil,dom_gross_mil,genres,avg_rating,staff,rating_bool,staff_bool
0,Toy Story 3,200.0,415.0,"Adventure,Animation,Comedy",8.00,10.0,False,False
1,Alice In Wonderland (2010),NaN,334.2,None,NaN,NaN,False,False
2,Harry Potter And The Deathly Hallows Part 1,NaN,296.0,"Adventure,Fantasy,Mystery",7.70,10.0,False,True
3,Inception,160.0,292.6,"Action,Adventure,Sci-Fi",8.55,10.0,False,True
4,Shrek Forever After,165.0,238.7,"Adventure,Animation,Comedy",6.20,10.0,False,True


In [21]:
q1 = """SELECT *
        FROM true_final_df
        WHERE rating_bool == False & staff_bool == False"""

actual_final_df = pysqldf(q1)
display(actual_final_df.head())
actual_final_df.shape

,title,prod_budget_mil,dom_gross_mil,genres,avg_rating,staff,rating_bool,staff_bool
0,Clash Of The Titans (2010),NaN,163.2,None,NaN,NaN,1,1
1,The Expendables,82.0,103.1,"Action,Adventure,Thriller",6.30,10.0,1,1
2,Grown Ups,75.0,162.0,Comedy,6.05,10.0,1,1
3,Knight & Day,NaN,76.4,None,NaN,NaN,1,1
4,The Social Network,40.0,97.0,"Biography,Drama",7.45,10.0,1,1


(7621, 8)

In [22]:
actual_final_df['rating_bool'] = actual_final_df['avg_rating'].isna()
actual_final_df['staff_bool'] = actual_final_df['staff'].isna()
actual_final_df.head()

,title,prod_budget_mil,dom_gross_mil,genres,avg_rating,staff,rating_bool,staff_bool
0,Clash Of The Titans (2010),NaN,163.2,None,NaN,NaN,True,True
1,The Expendables,82.0,103.1,"Action,Adventure,Thriller",6.30,10.0,False,False
2,Grown Ups,75.0,162.0,Comedy,6.05,10.0,False,False
3,Knight & Day,NaN,76.4,None,NaN,NaN,True,True
4,The Social Network,40.0,97.0,"Biography,Drama",7.45,10.0,False,False


In [23]:
q2 = """SELECT *
        FROM actual_final_df
        WHERE rating_bool == False"""

real_final_df = pysqldf(q2)
display(real_final_df.head())
real_final_df.shape

,title,prod_budget_mil,dom_gross_mil,genres,avg_rating,staff,rating_bool,staff_bool
0,The Expendables,82.0,103.1,"Action,Adventure,Thriller",6.30,10.0,0,0
1,Grown Ups,75.0,162.0,Comedy,6.05,10.0,0,0
2,The Social Network,40.0,97.0,"Biography,Drama",7.45,10.0,0,0
3,The Sorcerers Apprentice,160.0,63.2,"Action,Adventure,Family",6.00,10.0,0,0
4,Due Date,65.0,100.5,"Adventure,Comedy",6.40,10.0,0,0


(3555, 8)

In [24]:
real_final_df['avg_rating'].isna().value_counts()

False    3555
Name: avg_rating, dtype: int64

In [25]:
real_final_df['staff'].isna().value_counts()

False    3555
Name: staff, dtype: int64

In [26]:
real_final_df.drop(['rating_bool', 'staff_bool'], axis = 1, inplace = True)
real_final_df.head()

,title,prod_budget_mil,dom_gross_mil,genres,avg_rating,staff
0,The Expendables,82.0,103.1,"Action,Adventure,Thriller",6.30,10.0
1,Grown Ups,75.0,162.0,Comedy,6.05,10.0
2,The Social Network,40.0,97.0,"Biography,Drama",7.45,10.0
3,The Sorcerers Apprentice,160.0,63.2,"Action,Adventure,Family",6.00,10.0
4,Due Date,65.0,100.5,"Adventure,Comedy",6.40,10.0


In [27]:
for column in real_final_df.columns:
    print(f'{column.title()} has: {real_final_df[column].isna().sum()} NaN values out of {real_final_df.shape[0]}, for a fraction of {round(100*(real_final_df[column].isna().sum()/real_final_df.shape[0]), 3)}%.')

Title has: 0 NaN values out of 3555, for a fraction of 0.0%.
Prod_Budget_Mil has: 1505 NaN values out of 3555, for a fraction of 42.335%.
Dom_Gross_Mil has: 16 NaN values out of 3555, for a fraction of 0.45%.
Genres has: 52 NaN values out of 3555, for a fraction of 1.463%.
Avg_Rating has: 0 NaN values out of 3555, for a fraction of 0.0%.
Staff has: 0 NaN values out of 3555, for a fraction of 0.0%.


In [73]:
q3 = """SELECT prod_budget_mil, dom_gross_mil
        FROM real_final_df"""

to_be_scaled_df = pysqldf(q3)

x = to_be_scaled_df.values # Converting to an array instead of a dataframe
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
scaled_df = pd.DataFrame(x_scaled)
real_final_df['normal_budget'] = scaled_df[0]
real_final_df['normal_gross'] = scaled_df[1]
real_final_df['ease_of_make'] = (1 - real_final_df['normal_budget'])/real_final_df['staff']
real_final_df['normal_ease'] = ((real_final_df['ease_of_make'] - real_final_df['ease_of_make'].min()) / 
                                (real_final_df['ease_of_make'].max() - real_final_df['ease_of_make'].min()))
real_final_df.head()

,title,prod_budget_mil,dom_gross_mil,genres,avg_rating,staff,normal_budget,normal_gross,ease_of_make,normal_ease
0,The Expendables,82.0,103.1,"Action,Adventure,Thriller",6.30,10.0,0.192941,0.135569,0.080706,0.080725
1,Grown Ups,75.0,162.0,Comedy,6.05,10.0,0.176471,0.213018,0.082353,0.082372
2,The Social Network,40.0,97.0,"Biography,Drama",7.45,10.0,0.094118,0.127548,0.090588,0.090610
3,The Sorcerers Apprentice,160.0,63.2,"Action,Adventure,Family",6.00,10.0,0.376471,0.083103,0.062353,0.062368
4,Due Date,65.0,100.5,"Adventure,Comedy",6.40,10.0,0.152941,0.132150,0.084706,0.084726


In [81]:
label_dict = {'prod_budget_mil': 'Production Budget in Millions', 'dom_gross_mil':
               'Domestic Gross in Millions', 'staff': '(Assumed) Weighted Production Staff',
               'genres': 'Genres', 'normal_gross': 'Normalized Domestic Gross',
               'avg_rating': 'Average Rating', 'title': 'Title', 'normal_budget':
               'Normalized Budget', 'ease_of_make': 'How "Easy" was The Movie to Make?'}

fig = px.scatter(real_final_df, y = "prod_budget_mil", x = "staff", color = 'normal_gross',
                 hover_name = 'title', hover_data = ['genres', 'avg_rating', 'dom_gross_mil'],
                 title = 'Production Budget by Staff', labels = label_dict)

fig.update_layout(
    title={
        'y':0.9,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'}, showlegend = False)

fig.show()

In [82]:
for n in ('prod_budget_mil', 'staff'):
    fig = px.bar(real_final_df, x = n, y = 'ease_of_make', hover_name = 'title')
    display(fig.show())

None

None

In [83]:
real_final_df.head()

,title,prod_budget_mil,dom_gross_mil,genres,avg_rating,staff,normal_budget,normal_gross,ease_of_make,normal_ease
0,The Expendables,82.0,103.1,"Action,Adventure,Thriller",6.30,10.0,0.192941,0.135569,0.080706,0.080725
1,Grown Ups,75.0,162.0,Comedy,6.05,10.0,0.176471,0.213018,0.082353,0.082372
2,The Social Network,40.0,97.0,"Biography,Drama",7.45,10.0,0.094118,0.127548,0.090588,0.090610
3,The Sorcerers Apprentice,160.0,63.2,"Action,Adventure,Family",6.00,10.0,0.376471,0.083103,0.062353,0.062368
4,Due Date,65.0,100.5,"Adventure,Comedy",6.40,10.0,0.152941,0.132150,0.084706,0.084726


In [85]:
fig = px.scatter(real_final_df, x = 'ease_of_make', y = 'dom_gross_mil', color = 'normal_gross',
                hover_name = 'title', hover_data = ['staff', 'prod_budget_mil', 'genres'],
                size = 'avg_rating', labels = label_dict, title = 'Ease of Making a Movie by Domestic Gross')

fig.update_layout(
    title={
        'y':0.9,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'}, showlegend = False)

fig.show()